<a href="https://colab.research.google.com/github/DiMorten/FCN_ConvLSTM_Crop_Recognition_Open_Set/blob/coords5/train_and_evaluate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dependencies

In [1]:
!pip install icecream
#%tensorflow_version 1.x
import os
!pip install kora
from kora import drive
import time
!pip install colorama

ds_path='/content/drive/My Drive/PhD/datasets/cv_data/'

     |████████████████████████████████| 57 kB 4.1 MB/s 
     |████████████████████████████████| 56 kB 4.8 MB/s 


In [2]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
git_clone = True

if git_clone == True:
  os.chdir('/content')
  %rm -rf FCN_ConvLSTM_Crop_Recognition_Open_Set
  !git clone --branch coords5 https://github.com/DiMorten/FCN_ConvLSTM_Crop_Recognition_Open_Set.git

Cloning into 'FCN_ConvLSTM_Crop_Recognition_Open_Set'...
remote: Enumerating objects: 2189, done.
remote: Counting objects: 100% (2189/2189), done.
remote: Compressing objects: 100% (1618/1618), done.
remote: Total 2189 (delta 1440), reused 1062 (delta 366), pack-reused 0
Receiving objects: 100% (2189/2189), 37.52 MiB | 29.35 MiB/s, done.
Resolving deltas: 100% (1440/1440), done.


## Download images into proper folder

In [4]:
!cp -r /content/drive/MyDrive/PhD/datasets/cv_data /content/FCN_ConvLSTM_Crop_Recognition_Open_Set/dataset/dataset/

In [5]:
os.chdir('/content/FCN_ConvLSTM_Crop_Recognition_Open_Set/networks/convlstm_networks/train_src')
os.getcwd()
os.listdir()

['mosaic.py.old',
 'postprocessing.py',
 'model_input_mode.py',
 'model_best_UUnet4ConvLSTM_jun_cv_criteria_0_92',
 'densnet_timedistributed.py',
 '__init__.py',
 'keras_weighted_categorical_crossentropy.py',
 'deb.py',
 'model.py',
 'modelArchitecture.py',
 'main.py',
 'patch_extractor.py',
 'parameters',
 'mosaic.py',
 'metrics.py',
 'monitor.py',
 'dataset.py',
 'generator.py',
 'open_set.py',
 'obj',
 'dataSource.py',
 'densnet.py',
 'analysis']

In [6]:
from colorama import init
init()
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPool2D, Flatten, Dropout, Conv2DTranspose
# from tensorflow.keras.callbacks import ModelCheckpoint , EarlyStopping
from tensorflow.keras.optimizers import Adam,Adagrad 
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
import tensorflow.keras as keras

import numpy as np
from sklearn.utils import shuffle
import cv2
import argparse
import tensorflow as tf

from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras import metrics
import sys
import glob

from sklearn.metrics import confusion_matrix,f1_score,accuracy_score,classification_report
# Local
from densnet import DenseNetFCN
from densnet_timedistributed import DenseNetFCNTimeDistributed

#from metrics import fmeasure,categorical_accuracy
import deb
from keras_weighted_categorical_crossentropy import weighted_categorical_crossentropy, sparse_accuracy_ignoring_last_label, weighted_categorical_crossentropy_ignoring_last_label, categorical_focal_ignoring_last_label, weighted_categorical_focal_ignoring_last_label
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import ConvLSTM2D, UpSampling2D, multiply
from tensorflow.keras.regularizers import l1,l2
import time
import pickle
#from tensorflow.keras_self_attention import SeqSelfAttention
import pdb
import pathlib
from pathlib import Path, PureWindowsPath
from tensorflow.keras.layers import Conv3DTranspose, Conv3D

from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from collections import Counter


#from datagenerator import DataGenerator
from generator import DataGenerator, DataGeneratorWithCoords, DataGeneratorWithCoordsRandom

import matplotlib.pyplot as plt
sys.path.append('../../../dataset/dataset/patches_extract_script/')
from dataSource import DataSource, SARSource, OpticalSource, Dataset, LEM, LEM2, CampoVerde, OpticalSourceWithClouds, Humidity
from model_input_mode import MIMFixed, MIMVarLabel, MIMVarSeqLabel, MIMVarLabel_PaddedSeq, MIMFixedLabelAllLabels, MIMFixed_PaddedSeq
from parameters.parameters_reader import ParamsTrain

from icecream import ic
from monitor import Monitor, MonitorNPY, MonitorGenerator, MonitorNPYAndGenerator
import natsort
from model import NetModel, ModelFit, ModelLoadGeneratorWithCoords
from dataset import Dataset, DatasetWithCoords

from patch_extractor import PatchExtractor
ic.configureOutput(includeContext=False)
np.random.seed(2021)
tf.random.set_seed(2021)

from main import TrainTest

In [7]:
!nvidia-smi

Sun Aug 22 23:03:57 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Set parameters

Parameters can be modified in /content/FCN_ConvLSTM_Crop_Recognition_Open_Set/networks/convlstm_networks/train_src/parameters/parameters_reader.py

In [8]:
from pathlib import Path

paramsTrain = ParamsTrain('parameters/')
paramsTrain.mim = MIMFixed_PaddedSeq()

paramsTrain.getFullIms = True
paramsTrain.coordsExtract = True
paramsTrain.train = True

paramsTrain.train_overlap_percentage = 0
paramsTrain.trainGeneratorRandom = False
paramsTrain.patch_len = 32
paramsTrain.stride = int(paramsTrain.patch_len - paramsTrain.patch_len * paramsTrain.train_overlap_percentage)
paramsTrain.patch_step_train = paramsTrain.stride
paramsTrain.patch_step_test = paramsTrain.patch_len # to do: paramsTrain.getCalculatedParams() does these calculations

paramsTrain.dataset = 'cv'
paramsTrain.seq_date = 'jun'
paramsTrain.path = Path("../../../dataset/dataset/") / (paramsTrain.dataset + "_data")

paramsTrain.test_overlap_percentage = 0

paramsTrain.dataSource = SARSource()
paramsTrain.openSetMethod = None
trainTest = TrainTest(paramsTrain)

patchExtractor = PatchExtractor(paramsTrain, trainTest.ds)	

[@debug] parameters_reader.py:115 in __init__()- self.seq_date: 'mar'


self.known_classes [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]


[@debug] parameters_reader.py:174 in __init__()- self.stride: 32


['parameters_reader.py', 'save_nonaugmented_train_patches_unknownclasses.json', 'save_nonaugmented_train_patches.json', 'cv', 'twokkc_parameters_closedset_groupclasses.json', 'twokkc_parameters_openset.json', '__pycache__', 'twokkc_save_nonaugmented_train_patches.json', 'save_nonaugmented_train_patches_lessclass8.json', 'allkkc_save_nonaugmented_train_patches.json', 'params_batchprocessing.py', 'parameters_openset_lessclass8.json', '__init__.py', 'params_reconstruct.py', 'no_mode.json', 'allkkc_parameters_openset.json', 'parameters_openset.json', 'parameters_closedset_groupclasses.json', 'parameters_openset_specifyunknownclasses.json', 'parameters_closedset_groupclasses_lessclass8.json']
[@debug] self.seq_mode = fixed
[@debug] self.mim = <model_input_mode.MIMFixed_PaddedSeq object at 0x7fd425ecec10>
[@debug] self.ds = <dataSource.CampoVerde object at 0x7fd4264cab10>
20151029
20151110
20151122
20151204
20151216
20160121
20160214
20160309
20160321
20160508
20160520
20160613
dotys_sin_cos

[@debug] patch_extractor.py:17 in __init__()
         self.dataSource: <dataSource.SARSource object at 0x7fd425ff8e10>
[@debug] patch_extractor.py:26 in __init__()
         self.conf['path']/self.label_folder/"/": PosixPath('/')
[@debug] patch_extractor.py:35 in __init__()
         self.conf["in_npy_path"]: PosixPath('../../../dataset/dataset/cv_data/in_sar')
[@debug] patch_extractor.py:43 in __init__()
         self.conf["train"]["mask"]["dir"]: PosixPath('../../../dataset/dataset/cv_data/TrainTestMask.tif')
[@debug] patch_extractor.py:44 in __init__()
         os.getcwd(): '/content/FCN_ConvLSTM_Crop_Recognition_Open_Set/networks/convlstm_networks/train_src'


## Download or load sequence of images


In [9]:
if paramsTrain.getFullIms == True:
  patchExtractor.getFullIms()	
else:
  patchExtractor.fullImsLoad()


[@debug] patch_extractor.py:90 in getFullIms()
         patch["full_ims"].shape: (12, 8492, 7995, 2)
[@debug] patch_extractor.py:91 in getFullIms()
         self.dataset.im_list: ['20151029_S1',
                                '20151110_S1',
                                '20151122_S1',
                                '20151204_S1',
                                '20151216_S1',
                                '20160121_S1',
                                '20160214_S1',
                                '20160309_S1',
                                '20160321_S1',
                                '20160508_S1',
                                '20160520_S1',
                                '20160613_S1']


0 0
[@debug] conf["in_npy_path"]/(im_names[t_step]+".npy") = ../../../dataset/dataset/cv_data/in_sar/20151029_S1.npy
[@debug] patch["full_ims"].dtype = float16
[@debug] np.average(patch["full_ims"][t_step]) = 0.1871337890625
[@debug] np.max(patch["full_ims"][t_step]) = 1.0
[@debug] np.min(patch["full_ims"][t_step]) = 4.172325134277344e-07
../../../dataset/dataset/cv_data/labels/20151029_S1.tif
[@debug] conf["path"]/(self.dataSource.label_folder+"/"+label_names[t_step]+".tif") = ../../../dataset/dataset/cv_data/labels/20151029_S1.tif
[@debug] np.unique(patch["full_label_ims"][t_step],return_counts=True) = (array([ 0,  1,  2,  6,  7,  8,  9, 10, 11], dtype=int8), array([61778564,    45178,    51808,   131138,   438371,   155189,
        5136068,     1007,   156217]))
1 0
[@debug] conf["in_npy_path"]/(im_names[t_step]+".npy") = ../../../dataset/dataset/cv_data/in_sar/20151110_S1.npy
[@debug] patch["full_ims"].dtype = float16
[@debug] np.average(patch["full_ims"][t_step]) = 0.180419921875


[@debug] dataSource.py:369 in im_load()
         patch["full_ims"].shape: (12, 8492, 7995, 2)
[@debug] dataSource.py:370 in im_load()
         patch["full_label_ims"].shape: (12, 8492, 7995)
[@debug] dataSource.py:371 in im_load()
         patch["full_ims"].dtype: dtype('float16')
[@debug] dataSource.py:372 in im_load()
         patch["full_label_ims"].dtype: dtype('int8')


[@debug] np.unique(patch['full_label_ims'],return_counts=True) = (array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11], dtype=int8), array([741342768,  17247080,   7809932,  12457060,    185790,     51702,
         2462898,   5372125,   1862268,  24044169,     12084,   1874604]))
0.0 1.0 0.1702
[@debug] self.dataset.name = cv
[@debug] self.dataset.scaler_name = cv
[@debug] self.dataset.seq_mode = fixed
[@debug] self.dataset.seq_date = jun
[@debug] self.dataset.scaler_load = False
[@debug] im.shape = (12, 8492, 7995, 2)
[@debug] im_flat[mask_flat==1,:].shape = (38664648, 2)
0.0001407 1.0 0.0779
[@debug] im_norm.shape = (12, 8492, 7995, 2)
FINISHED NORMALIZING, RESULT:
-1.432 51.53 3.117
[@debug] im.shape = (12, 8492, 7995)
[@debug] mask_train.shape = (8492, 7995)
[@debug] im.dtype = uint8
[@debug] mask_train.dtype = uint8
[@debug] im_train.shape = (12, 8492, 7995)
Train masked unique/count [ 0  1  2  3  4  5  6  7  8  9 10 11] [776057832   9399025   4170646   6930141    103494     346

[@debug] patch_extractor.py:127 in getFullIms()
         self.paramsTrain.path / 'full_ims/full_ims_test.npy': PosixPath('../../../dataset/dataset/cv_data/full_ims/full_ims_test.npy')


## Extract coords of image patches

In [10]:

if paramsTrain.coordsExtract == True:
  patchExtractor.extract()

del patchExtractor



STARTED PATCH EXTRACTION
[@debug] gridx.shape = (250,)
[@debug] gridy.shape = (266,)


[@debug] patch_extractor.py:205 in extract()
         coords_train.shape: (4983, 2)
         coords_test.shape: (4626, 2)
[@debug] patch_extractor.py:206 in extract()
         coords_train.dtype: dtype('int64')
[@debug] patch_extractor.py:207 in extract()
         coords_train[0]: array([7568,  656])


## Train


In [ ]:
	trainTest.setData()


	trainTest.preprocess(paramsTrain.model_name_id) # move into if

	trainTest.setModel()

	if paramsTrain.train == True:
		trainTest.train()
	else:
		trainTest.modelLoad(paramsTrain.model_name_id)

Initializing object...
12 2
[@debug] self.channel_n = 2


[@debug] dataset.py:96 in __init__()- self.class_n: 10


[@debug] self.t_len = 12
Initializing Dataset instance


[@debug] main.py:118 in setData()- self.data.class_n: 10
[@debug] main.py:146 in preprocess()
         self.model_name: '../results/convlstm_results/model/lm/model_best_UUnetConvLSTM_mar_lm_nomode.h5'
[@debug] dataset.py:186 in create_load()
         os.path.dirname(os.path.abspath(__file__)): '/content/FCN_ConvLSTM_Crop_Recognition_Open_Set/networks/convlstm_networks/train_src'
[@debug] dataset.py:187 in create_load()
         os.getcwd(): '/content/FCN_ConvLSTM_Crop_Recognition_Open_Set/networks/convlstm_networks/train_src'
[@debug] dataset.py:191 in create_load()
         self.patches['train']['coords'].shape: (4983, 2)
[@debug] dataset.py:284 in labelPreprocess()
         np.unique(self.full_label_train, return_counts=True): (array([ 0,  2,  3,  4,  6,  7,  8,  9, 10, 11], dtype=uint8),
                                                                array([64671486,   710232,  1374508,    31550,   136800,   212701,
                                                                   

[@debug] np.unique(self.full_label_train, return_counts=True) = (array([ 0,  2,  3,  4,  6,  7,  8,  9, 10, 11], dtype=uint8), array([64671486,   710232,  1374508,    31550,   136800,   212701,
          71804,   617076,      699,    66684]))


...

 — val_f1: [85.11 96.29 94.61 76.37 90.4  94.09 85.93  0.   77.97]
 — val_precision: [91.48 97.75 90.78 62.85 90.61 90.3  82.63  0.   71.5 ]
 — val_recall: [79.57 94.87 98.78 97.29 90.19 98.2  89.5   0.   85.72]
 — mean_f1: 77.86333333333333
oa 90.21
Found best weights at epoch 9
393/393 [==============================] - 36s 93ms/step - batch: 196.0000 - size: 16.0000 - loss: 0.0532 - accuracy: 0.1593 - val_loss: 0.0582 - val_accuracy: 0.1880 - mean_f1: 77.8633
Epoch 10/70
393/393 [==============================] - ETA: 0s - batch: 196.0000 - size: 16.0000 - loss: 0.0496 - accuracy: 0.1603

[@debug] generator.py:192 in __len__()- n_batches: 46
[@debug] generator.py:192 in __len__()- n_batches: 46
[@debug] generator.py:192 in __len__()- n_batches: 46


 — val_f1: [86.48 96.63 94.61 77.72 90.25 92.04 87.96  0.   79.4 ]
 — val_precision: [91.82 96.94 90.54 64.55 90.66 86.43 87.39  0.   74.48]
 — val_recall: [81.72 96.32 99.05 97.66 89.84 98.42 88.54  0.   85.02]
 — mean_f1: 78.34333333333333
oa 91.12
Found best weights at epoch 10
393/393 [==============================] - 37s 93ms/step - batch: 196.0000 - size: 16.0000 - loss: 0.0496 - accuracy: 0.1603 - val_loss: 0.0553 - val_accuracy: 0.1875 - mean_f1: 78.3433
Epoch 11/70
393/393 [==============================] - ETA: 0s - batch: 196.0000 - size: 16.0000 - loss: 0.0454 - accuracy: 0.1583

[@debug] generator.py:192 in __len__()- n_batches: 46
[@debug] generator.py:192 in __len__()- n_batches: 46
[@debug] generator.py:192 in __len__()- n_batches: 46


 — val_f1: [87.27 96.78 93.04 81.02 91.55 93.16 87.74  0.   80.23]
 — val_precision: [92.99 97.87 87.95 69.66 90.66 88.57 84.4   0.   79.07]
 — val_recall: [82.21 95.71 98.75 96.83 92.47 98.26 91.35  0.   81.42]
 — mean_f1: 78.97666666666666
oa 91.56
Found best weights at epoch 11
393/393 [==============================] - 36s 92ms/step - batch: 196.0000 - size: 16.0000 - loss: 0.0454 - accuracy: 0.1583 - val_loss: 0.0514 - val_accuracy: 0.1884 - mean_f1: 78.9767
Epoch 12/70
163/393 [===========>..................] - ETA: 18s - batch: 81.0000 - size: 16.0000 - loss: 0.0426 - accuracy: 0.1540

## Evaluate


In [ ]:
paramsMosaic = ParamsReconstruct(paramsTrain)

paramsMosaic.mosaic_flag = True

trainTest.evaluate(paramsMosaic)
